## Feature selection

### Importing libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

### Reading dataset

In [3]:
data = pd.read_csv(r"C:\Users\bhavv\Downloads\Final_Project_dataset_V2.csv")

### Dropping nulls from dataset and checking the data

In [4]:
data = data.dropna()

In [5]:
data

Case no. Date of notification to WHO (yyyy/mm/dd) Reporting country  \
0          1.0                               2017-08-15      Saudi Arabia   
1          2.0                               2017-08-16      Saudi Arabia   
2          3.0                               2017-08-18      Saudi Arabia   
3          4.0                               2017-08-21      Saudi Arabia   
4          5.0                               2017-08-24      Saudi Arabia   
...        ...                                      ...               ...   
2433       4.0                               2017-01-04      Saudi Arabia   
2434       5.0                               2017-01-04      Saudi Arabia   
2435       6.0                               2017-01-04      Saudi Arabia   
2437       8.0                               2017-01-02      Saudi Arabia   
2438       9.0                               2017-01-02      Saudi Arabia   

     City of residence   Age Sex Health care worker Comorbidities  \
0             Madinah   67.0   M                 No           Yes   
1               Jeddah  69.0   M                 No           Yes   
2               Jeddah  51.0   M                 No           Yes   
3               Riyadh  48.0   M                 No           Yes   
4      Dawmet Aljandal  40.0   F                 No           Yes   
...                ...   ...  ..                ...           ...   
2433           Buridah  88.0   F                 No           Yes   
2434           Buridah  87.0   F                 No           Yes   
2435            Riyadh  56.0   M                 No           Yes   
2437           Madinah  70.0   M                 No           Yes   
2438           Buridah  63.0   F                 No           Yes   

     Exposure to camels Camel milk consumption Exposure to MERS-CoV cases  \
0                   Yes                     No                         No   
1                    No                     No                         No   
2                    No                     No                         No   
3                   Yes                    Yes                         No   
4                    No                     No                         No   
...                 ...                    ...                        ...   
2433                 No                     No                         No   
2434                 No                     No                         No   
2435                 No                     No                         No   
2437                Yes                    Yes                         No   
2438                 No                     No                         No   

     Date of symptoms onset (yyyy/mm/dd)  \
0                             2017-08-12   
1                             2017-08-13   
2                             2017-08-11   
3                             2017-08-18   
4                             2017-08-17   
...                                  ...   
2433                          2016-12-19   
2434                          2016-12-29   
2435                          2016-12-28   
2437                          2016-12-25   
2438                          2016-12-29   

     Date of first hospitalization (yyyy/mm/dd)  \
0                                    2017-08-14   
1                                    2017-08-15   
2                                    2017-08-17   
3                                    2017-08-20   
4                                    2017-08-21   
...                                         ...   
2433                                 2017-01-01   
2434                                 2017-01-02   
2435                                 2017-01-02   
2437                                 2017-01-01   
2438                                 2016-12-31   

     Date of laboratory confirmation (yyyy/mm/dd) Day_diff    Status  
0                                      2017-08-14        2     Alive  
1                                      2017

### Performing label encoding for Random Forest Feature Selection

In [6]:

for column in data.columns:
    if data[column].dtype == type(object):
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))


### Splitting data into X and y

In [7]:
X = data.iloc[:,0:15]
y = data["Status"]


### Modelling Random Forest for Feature Selection

In [8]:
randomforest = RandomForestRegressor(random_state=0)

model = randomforest.fit(X,y)

from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(model,threshold=0.05)
sfm.fit(X,y)
for feature_list_index in sfm.get_support(indices=True):
    print(X.columns[feature_list_index])



C:\Users\bhavv\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Case no.
Date of notification to WHO (yyyy/mm/dd)
City of residence
Age
Date of symptoms onset (yyyy/mm/dd)
Date of first hospitalization (yyyy/mm/dd)
Date of laboratory confirmation (yyyy/mm/dd)
Day_diff


### Storing all sorted data in a dataframe result_RF

In [9]:
result_RF = pd.DataFrame(list(zip(X.columns,model.feature_importances_)), columns = ['predictor','Gini coefficient'])

In [10]:
result_RF.sort_values(by='Gini coefficient', ascending=False)

predictor  Gini coefficient
4                                            Age          0.198117
0                                       Case no.          0.140378
3                              City of residence          0.133383
1       Date of notification to WHO (yyyy/mm/dd)          0.090779
12    Date of first hospitalization (yyyy/mm/dd)          0.088125
11           Date of symptoms onset (yyyy/mm/dd)          0.085896
14                                      Day_diff          0.083865
13  Date of laboratory confirmation (yyyy/mm/dd)          0.074523
5                                            Sex          0.033264
7                                  Comorbidities          0.027898
2                              Reporting country          0.014515
8                             Exposure to camels          0.013073
10                    Exposure to MERS-CoV cases          0.007775
9                         Camel milk consumption          0.004866
6                             Health care worker          0.003546

### Performing Standarization for Lasso

In [11]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)


### Modeling Lasso for Feature Selection

In [12]:
model_l = Lasso(alpha=0.01,positive=True)
model_l.fit(X_std,y)


Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=True, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

### Storing all sorted data in a dataframe result_Lasso

In [13]:
result_Lasso = pd.DataFrame(list(zip(X.columns,model_l.coef_)), columns = ['predictor','coefficient'])

In [14]:
result_Lasso.sort_values(by='coefficient', ascending=False)

predictor  coefficient
4                                            Age     0.107540
2                              Reporting country     0.082143
7                                  Comorbidities     0.080815
5                                            Sex     0.061456
10                    Exposure to MERS-CoV cases     0.055873
3                              City of residence     0.037822
0                                       Case no.     0.000000
1       Date of notification to WHO (yyyy/mm/dd)     0.000000
6                             Health care worker     0.000000
8                             Exposure to camels     0.000000
9                         Camel milk consumption     0.000000
11           Date of symptoms onset (yyyy/mm/dd)     0.000000
12    Date of first hospitalization (yyyy/mm/dd)     0.000000
13  Date of laboratory confirmation (yyyy/mm/dd)     0.000000
14                                      Day_diff     0.000000